# This code will extract the longest open reading fasta outputs form the pep file created in transdecoder. Then the file is used in CD hit to find if any of the sequences are 90% similar to each other.

In [1]:
from collections import defaultdict
import pandas as pd
from os.path import exists, join
from Bio import SeqIO
from pycdhit import cd_hit, read_clstr
import pdb
import os
import re

Import all the files needed

In [84]:
input_longest_orf_dir = "../input/cnidaria/longest_orfs/transdecoder_part1/S_pistillata_longest_orfs.pep"

input_dir = "../input/"

seq_id_file = "../input/cnidaria/S_pistillata_tir_sig.txt"

output_file_parsed = "../output/cnidaria/parsed_sig_pep/parsed_sig_pep_results_S_pistillata.fasta"
output_dir_cdhit = "../output/cnidaria/cdhit/S_pistillata_cdhit_out"
output_file_cdhit = "../output/cnidaria/cdhit/S_pistillata_cdhit_out.clstr"
output_dir = "../output/parsed_files"



required = [input_dir,input_longest_orf_dir,output_dir]

for r in required:
    if not exists(r):
        raise FileNotFoundError(f"The specified hmmer file {r} does not seem to be in the specified path.")
    print(f"Confirmed that required file or directory {r} exists.")

Confirmed that required file or directory ../input/ exists.
Confirmed that required file or directory ../input/cnidaria/longest_orfs/transdecoder_part1/S_pistillata_longest_orfs.pep exists.
Confirmed that required file or directory ../output/parsed_files exists.


We are using Biopython to parse the significant sequences out of the orginal pep file and placing them into a new file.

In [75]:
#open files
pep_file = open(input_longest_orf_dir, 'r')
select_seq_out = open(output_file_parsed, 'w')

In [76]:
my_sequences = []
with open(seq_id_file) as seq_ids:
    for line in seq_ids:
        my_sequences.extend(line.split(' '))

In [77]:
for record in SeqIO.parse(pep_file, 'fasta'):
    for items in my_sequences:
        if items.strip() == record.id:
            select_seq_out.write(">" + record.id + "\n")
            select_seq_out.write(str(record.seq) + "\n")
            
pep_file.close()
select_seq_out.close

<function TextIOWrapper.close()>

# Run CD-Hit

In [85]:
#The file needs to be reopened as there is an error since its open with a 'w' in the previous code section!
seq_out = output_file_parsed

In [86]:
# run cd-hit on the records
res = cd_hit(
    i=seq_out,
    o=output_dir_cdhit,
    c=0.9,
    d=0,
    sc=1
)
df_clstr = read_clstr(output_file_cdhit)